In [2]:
import numpy as np

def ortho_Rpp(cU, cL,rhoU, rhoL, chiU, chiL, phi, tht):
    
    temp, rpd = np.pi/180.
    chiU = chiU*rpd
    chiL = chiL*rpd
    phi = phi*rpd
    tht = tht*rpd
    cphi = np.cos(phi)
    sphi = np.sin(phi)
    stht = np.sin(tht)
    ctht = np.cos(tht)
    n1 = cphi*stht
    n2 = sphi*stht
    n3 = ctht

    # Rotate by chi
    cU = axial_rotation(chiU,cU)
    c11Ur=cU[0][0]
    c12Ur=cU[0][1]
    c13Ur=cU[0][2]
    c16Ur=cU[0][5]
    c22Ur=cU[1][1]
    c23Ur=cU[1][2]
    c26Ur=cU[1][5]
    c33Ur=cU[2][2]
    c36Ur=cU[2][5]
    c44Ur=cU[3][3]
    c45Ur=cU[3][4]
    c55Ur=cU[4][4]
    c66Ur=cU[5][5]
    # System.out.println("cU: "+Arrays.deepToString(cU));

    cL = axial_rotation(chiL,cL)
    c11Lr=cL[0][0]
    c12Lr=cL[0][1]
    c13Lr=cL[0][2]
    c16Lr=cL[0][5]
    c22Lr=cL[1][1]
    c23Lr=cL[1][2]
    c26Lr=cL[1][5]
    c33Lr=cL[2][2]
    c36Lr=cL[2][5]
    c44Lr=cL[3][3]
    c45Lr=cL[3][4]
    c55Lr=cL[4][4]
    c66Lr=cL[5][5]

    # Form the velocity form of the Christoffel matrix for the upper layer
    CM=np.array([[c11Ur*n1*n1 + c66Ur*n2*n2 + c55Ur*n3*n3 + 2*c16Ur*n1*n2,
                  c16Ur*n1*n1 + c26Ur*n2*n2 + c45Ur*n3*n3 + (c12Ur+c66Ur)*n1*n2,
                  (c13Ur+c55Ur)*n1*n3 + (c36Ur+c45Ur)*n2*n3],
                 [c16Ur*n1*n1 + c26Ur*n2*n2 + c45Ur*n3*n3 + (c12Ur+c66Ur)*n1*n2,
                  c66Ur*n1*n1 + c22Ur*n2*n2 + c44Ur*n3*n3 + 2*c26Ur*n1*n2,
                  (c36Ur+c45Ur)*n1*n3 + (c23Ur+c44Ur)*n2*n3],
                 [(c13Ur+c55Ur)*n1*n3 + (c36Ur+c45Ur)*n2*n3,
                  (c36Ur+c45Ur)*n1*n3 + (c23Ur+c44Ur)*n2*n3,
                  c55Ur*n1*n1+c44Ur*n2*n2+c33Ur*n3*n3]])
    CM = CM/rhoU
    
    #System.out.println("CM: "+Arrays.deepToString(CM))

    # Find eigenvalues (phase velocities) and eigenvectors (phase polarization
    # vectors) from the Christoffel matrix just obtained, and use to create the
    # slowness vector for the incident wave
    temp3 = eigval3symm(CM)
    #System.out.println("temp3: "+Arrays.toString(temp3));

    # double Vinc = Math.sqrt(temp3[1-1]); // this assumes an incident T-wave
    # double Vinc = Math.sqrt(temp3[2-1]); // this assumes an incident S-wave
    Vinc = np.sqrt(temp3[2]) # this assumes an incident P-wave

    s1 = n1/Vinc
    s2 = n2/Vinc
    s3inc = n3/Vinc

    # Generate coefficients for bicubic for upper layer
    # [a3 a2 a1 a0]=bicubic_coeffs_orthotropic(s1,s2,rhoL,c11L,c12L,c13L,c22L,c23L,c33L,c44L,c55L,c66L);
    temp4=bicubic_coeffs_monoclinic(s1,s2,rhoU,c11Ur,c12Ur,c13Ur,c16Ur,c22Ur,c23Ur,c26Ur,c33Ur,c36Ur,c44Ur,c45Ur,c55Ur,c66Ur)

    # Now solve the bicubic
    num_roots = cubic_num_real_roots(temp4[3],temp4[2],temp4[1],temp4[0])
    if num_roots == 1:
        print("Guess what, you only have one real root in your bicubic (upper)")
        temp3 = cubic_three_realified_roots(temp4[3],temp4[2],temp4[1],temp4[0])
    if num_roots == 3:
        temp3 = cubic_three_real_roots(temp4[3],temp4[2],temp4[1],temp4[0])

    # Assign roots
    temp3 = order3incr(temp3)
    s3U2P = temp3[0]
    s3U2S = temp3[1]
    s3U2T = temp3[2]
    # Form vertical slownesses
    # HERE - could have imaginary vertical slownesses 
    s3UP = np.sqrt(s3U2P)
    s3US = np.sqrt(s3U2S)
    s3UT = np.sqrt(s3U2T)
    # Form the total slownesses
    #double sU2P = s1*s1+s2*s2+s3U2P;
    #double sU2S = s1*s1+s2*s2+s3U2S;
    #double sU2T = s1*s1+s2*s2+s3U2T;
    #double sUP = Math.sqrt(sU2P);
    #double sUS = Math.sqrt(sU2S);
    #double sUT = Math.sqrt(sU2T);
    # Now we can form the velocities of the three reflected waves
    #double vUP = 1.0/sUP
    #double vUS = 1.0/sUS
    #double vUT = 1.0/sUT
    # Check that reflected P-wave has same velocity as incident wave
    #double  vUP, Vinc
    # Check that reflected P-wave has same angles as incident wave
    #temp1=s1*vUP, n1
    #temp2=s2*vUP, n2

    # Generate coefficients for bicubic for lower layer
    #[a3 a2 a1 a0]=bicubic_coeffs_orthotropic(s1,s2,rhoL,c11L,c12L,c13L,c22L,c23L,c33L,c44L,c55L,c66L);
    temp4=bicubic_coeffs_monoclinic(s1,s2,rhoL,c11Lr,c12Lr,c13Lr,c16Lr,c22Lr,c23Lr,c26Lr,c33Lr,c36Lr,c44Lr,c45Lr,c55Lr,c66Lr)

    # Now solve the bicubic
    #num_roots = cubic_num_real_roots(temp4[0],temp4[1],temp4[2],temp4[3]);
    num_roots = cubic_num_real_roots(temp4[3],temp4[2],temp4[1],temp4[0])
    if num_roots == 1:
        print("Guess what, you only have one real root in your bicubic (upper)")
        temp3 = cubic_three_realified_roots(temp4[3],temp4[2],temp4[1],temp4[0])
    if (num_roots == 3):
        temp3 = cubic_three_real_roots(temp4[3],temp4[2],temp4[1],temp4[0])

    # Assign roots
    temp3 = order3incr(temp3);
    s3L2P = temp3[0]
    s3L2S = temp3[1]
    s3L2T = temp3[2]
    # Form vertical slownesses
    # HERE - could have imaginary vertical slownesses 
    s3LP = Math.sqrt(s3L2P)
    s3LS = Math.sqrt(s3L2S)
    s3LT = Math.sqrt(s3L2T)
    # Form the total slownesses
    #double sL2P = s1*s1+s2*s2+s3L2P;
    #double sL2S = s1*s1+s2*s2+s3L2S;
    #double sL2T = s1*s1+s2*s2+s3L2T;
    #double sLP = Math.sqrt(sL2P);
    #double sLS = Math.sqrt(sL2S);
    #double sLT = Math.sqrt(sL2T);
    # Now we can form the velocities of the three reflected waves
    #double vLP = 1.0/sLP
    #double vLS = 1.0/sLS
    #double vLT = 1.0/sLT


    # Form the velocity form of the Christoffel matrix for the reflected P-wave
    CMUP = np.array([[c11Ur*s1*s1 + c66Ur*s2*s2 + c55Ur*s3U2P + 2*c16Ur*s1*s2,
                      c16Ur*s1*s1 + c26Ur*s2*s2 + c45Ur*s3U2P + (c12Ur+c66Ur)*s1*s2,
                      (c13Ur+c55Ur)*s1*s3UP + (c36Ur+c45Ur)*s2*s3UP],
                     [c16Ur*s1*s1 + c26Ur*s2*s2 + c45Ur*s3U2P + (c12Ur+c66Ur)*s1*s2,
                      c66Ur*s1*s1 + c22Ur*s2*s2 + c44Ur*s3U2P + 2*c26Ur*s1*s2,
                      (c36Ur+c45Ur)*s1*s3UP + (c23Ur+c44Ur)*s2*s3UP],
                     [(c13Ur+c55Ur)*s1*s3UP + (c36Ur+c45Ur)*s2*s3UP,
                      (c36Ur+c45Ur)*s1*s3UP + (c23Ur+c44Ur)*s2*s3UP,
                      c55Ur*s1*s1+c44Ur*s2*s2+c33Ur*s3U2P]])

    # HERE - could have imaginary vertical slowness above - affects next step?
    eigUP = eigvec3symm(CMUP,rhoU)
    temp1 = np.abs(np.sqrt(eigUP[0]*eigUP[0] + eigUP[1]*eigUP[1] + eigUP[2]*eigUP[2]))
    temp1 = (eigUP[0]*s1+eigUP[1]*s2)/np.abs(eigUP[0]*s1+eigUP[1]*s2) / temp1

    eigUP = eigUP*temp1

    # Form the velocity form of the Christoffel matrix for the reflected S-wave
    CMUS = np.array([[c11Ur*s1*s1 + c66Ur*s2*s2 + c55Ur*s3U2S + 2*c16Ur*s1*s2,
                      c16Ur*s1*s1 + c26Ur*s2*s2 + c45Ur*s3U2S + (c12Ur+c66Ur)*s1*s2,
                      (c13Ur+c55Ur)*s1*s3US + (c36Ur+c45Ur)*s2*s3US],
                     [c16Ur*s1*s1 + c26Ur*s2*s2 + c45Ur*s3U2S + (c12Ur+c66Ur)*s1*s2,
                      c66Ur*s1*s1 + c22Ur*s2*s2 + c44Ur*s3U2S + 2*c26Ur*s1*s2,
                      (c36Ur+c45Ur)*s1*s3US + (c23Ur+c44Ur)*s2*s3US],
                     [(c13Ur+c55Ur)*s1*s3US + (c36Ur+c45Ur)*s2*s3US,
                      (c36Ur+c45Ur)*s1*s3US + (c23Ur+c44Ur)*s2*s3US,
                      c55Ur*s1*s1+c44Ur*s2*s2+c33Ur*s3U2S]])

    eigUS = eigvec3symm(CMUS,rhoU)
    temp1 = np.abs(np.sqrt( eigUS[0]*eigUS[0] + eigUS[1]*eigUS[1] + eigUS[2]*eigUS[2]))
    temp1 = (eigUS[0]*s1+eigUS[1]*s2)/np.abs(eigUS[0]*s1+eigUS[1]*s2) / temp1;

    eigUS = eigUS*temp1;

    # Form the velocity form of the Christoffel matrix for the reflected T-wave
    CMUT = np.array([[c11Ur*s1*s1 + c66Ur*s2*s2 + c55Ur*s3U2T + 2*c16Ur*s1*s2,
                      c16Ur*s1*s1 + c26Ur*s2*s2 + c45Ur*s3U2T + (c12Ur+c66Ur)*s1*s2,
                      (c13Ur+c55Ur)*s1*s3UT + (c36Ur+c45Ur)*s2*s3UT],
                     [c16Ur*s1*s1 + c26Ur*s2*s2 + c45Ur*s3U2T + (c12Ur+c66Ur)*s1*s2,
                      c66Ur*s1*s1 + c22Ur*s2*s2 + c44Ur*s3U2T + 2*c26Ur*s1*s2,
                      (c36Ur+c45Ur)*s1*s3UT + (c23Ur+c44Ur)*s2*s3UT],
                     [(c13Ur+c55Ur)*s1*s3UT + (c36Ur+c45Ur)*s2*s3UT,
                      (c36Ur+c45Ur)*s1*s3UT + (c23Ur+c44Ur)*s2*s3UT,
                      c55Ur*s1*s1+c44Ur*s2*s2+c33Ur*s3U2T]])

    eigUT = eigvec3symm(CMUT,rhoU)
    temp1 = np.abs(np.sqrt( eigUT[0]*eigUT[0] + eigUT[1]*eigUT[1] + eigUT[2]*eigUT[2] ))
    temp1 = (eigUT[0]*s1+eigUT[1]*s2)/np.abs(eigUT[0]*s1+eigUT[1]*s2) / temp1

    eigUT = eigUT*temp1

    # try and match up qSV and qSH with the most appropriate eigenvalues/eigenvectors
    temp1 = eigUS[0]*n3 + eigUS[1]*n2 - eigUS[2]*n1
    temp  = eigUT[0]*n3 + eigUT[1]*n2 - eigUT[2]*n1

    if (temp > temp1):
        temp1 = s3US
        s3US = s3UT
        s3UT = temp1
    
        temp3 = eigUS
        eigUS = eigUT
        eigUT = temp3

    # Form the velocity form of the Christoffel matrix for the transmitted P-wave
    CMLP = np.array([[c11Lr*s1*s1 + c66Lr*s2*s2 + c55Lr*s3L2P + 2*c16Lr*s1*s2,
                      c16Lr*s1*s1 + c26Lr*s2*s2 + c45Lr*s3L2P + (c12Lr+c66Lr)*s1*s2,
                      (c13Lr+c55Lr)*s1*s3LP + (c36Lr+c45Lr)*s2*s3LP],
                     [c16Lr*s1*s1 + c26Lr*s2*s2 + c45Lr*s3L2P + (c12Lr+c66Lr)*s1*s2,
                      c66Lr*s1*s1 + c22Lr*s2*s2 + c44Lr*s3L2P + 2*c26Lr*s1*s2,
                      (c36Lr+c45Lr)*s1*s3LP + (c23Lr+c44Lr)*s2*s3LP],
                     [(c13Lr+c55Lr)*s1*s3LP + (c36Lr+c45Lr)*s2*s3LP,
                      (c36Lr+c45Lr)*s1*s3LP + (c23Lr+c44Lr)*s2*s3LP,
                      c55Lr*s1*s1+c44Lr*s2*s2+c33Lr*s3L2P]])

    eigLP = eigvec3symm(CMLP,rhoL)
    temp1 = np.abs(np.sqrt( eigLP[0]*eigLP[0] + eigLP[1]*eigLP[1] + eigLP[2]*eigLP[2] ))
    temp1 = (eigLP[0]*s1+eigLP[1]*s2)/np.abs(eigLP[0]*s1+eigLP[1]*s2) / temp1

    eigLP = eigLP*temp1

    # Form the velocity form of the Christoffel matrix for the transmitted S-wave
    CMLS = np.array([[c11Lr*s1*s1 + c66Lr*s2*s2 + c55Lr*s3L2S + 2*c16Lr*s1*s2,
                      c16Lr*s1*s1 + c26Lr*s2*s2 + c45Lr*s3L2S + (c12Lr+c66Lr)*s1*s2,
                      (c13Lr+c55Lr)*s1*s3LS + (c36Lr+c45Lr)*s2*s3LS],
                      [c16Lr*s1*s1 + c26Lr*s2*s2 + c45Lr*s3L2S + (c12Lr+c66Lr)*s1*s2,
                       c66Lr*s1*s1 + c22Lr*s2*s2 + c44Lr*s3L2S + 2*c26Lr*s1*s2,
                       (c36Lr+c45Lr)*s1*s3LS + (c23Lr+c44Lr)*s2*s3LS],
                       [(c13Lr+c55Lr)*s1*s3LS + (c36Lr+c45Lr)*s2*s3LS,
                        (c36Lr+c45Lr)*s1*s3LS + (c23Lr+c44Lr)*s2*s3LS,
                        c55Lr*s1*s1+c44Lr*s2*s2+c33Lr*s3L2S]])

    eigLS = eigvec3symm(CMLS,rhoL)
    temp1 = np.abs(np.sqrt( eigLS[0]*eigLS[0] + eigLS[1]*eigLS[1] + eigLS[2]*eigLS[2] ))
    temp1 = (eigLS[0]*s1+eigLS[1]*s2)/np.abs(eigLS[0]*s1+eigLS[1]*s2) / temp1

    eigLS = eigLS*temp1

    # Form the velocity form of the Christoffel matrix for the transmitted T-wave
    CMLT = np.array([[c11Lr*s1*s1 + c66Lr*s2*s2 + c55Lr*s3L2T + 2*c16Lr*s1*s2,
                      c16Lr*s1*s1 + c26Lr*s2*s2 + c45Lr*s3L2T + (c12Lr+c66Lr)*s1*s2,
                      (c13Lr+c55Lr)*s1*s3LT + (c36Lr+c45Lr)*s2*s3LT],
                      [c16Lr*s1*s1 + c26Lr*s2*s2 + c45Lr*s3L2T + (c12Lr+c66Lr)*s1*s2,
                       c66Lr*s1*s1 + c22Lr*s2*s2 + c44Lr*s3L2T + 2*c26Lr*s1*s2,
                       (c36Lr+c45Lr)*s1*s3LT + (c23Lr+c44Lr)*s2*s3LT],
                      [(c13Lr+c55Lr)*s1*s3LT + (c36Lr+c45Lr)*s2*s3LT,
                       (c36Lr+c45Lr)*s1*s3LT + (c23Lr+c44Lr)*s2*s3LT,
                       c55Lr*s1*s1+c44Lr*s2*s2+c33Lr*s3L2T]])

    eigLT = eigvec3symm(CMLT,rhoL)
    temp1 = np.abs(np.sqrt( eigLT[0]*eigLT[0] + eigLT[1]*eigLT[1] + eigLT[2]*eigLT[2] ))
    temp1 = (eigLT[0]*s1+eigLT[1]*s2)/np.abs(eigLT[0]*s1+eigLT[1]*s2) / temp1

    eigLT = eigLT*temp1
 
    # try and match up qSV and qSH with the most appropriate eigenvalues/eigenvectors
    temp1 = eigLS[0]*n3 + eigLS[1]*n2 - eigLS[2]*n1
    temp2 = eigLT[0]*n3 + eigLT[1]*n2 - eigLT[2]*n1

    if (temp2 > temp1):
        temp = s3LS
        s3LS = s3LT
        s3LT = temp
    
        temp3 = eigLS
        eigLS = eigLT
        eigLT = temp3

    # Construct X and Y matrices

    XU = np.array([[eigUP[0], eigUS[0], eigUT[0]],
                   [eigUP[1], eigUS[1], eigUT[1]],
                   [-(c13Ur*eigUP[0]+c36Ur*eigUP[1])*s1-(c23Ur*eigUP[1]+c36Ur*eigUP[0])*s2-c33Ur*eigUP[2]*s3UP,
                    -(c13Ur*eigUS[0]+c36Ur*eigUS[1])*s1-(c23Ur*eigUS[1]+c36Ur*eigUS[0])*s2-c33Ur*eigUS[2]*s3US,
                    -(c13Ur*eigUT[0]+c36Ur*eigUT[1])*s1-(c23Ur*eigUT[1]+c36Ur*eigUT[0])*s2-c33Ur*eigUT[2]*s3UT]])

    YU = np.array([[-(c55Ur*s1+c45Ur*s2)*eigUP[2]-(c55Ur*eigUP[0]+c45Ur*eigUP[1])*s3UP,
                    -(c55Ur*s1+c45Ur*s2)*eigUS[2]-(c55Ur*eigUS[0]+c45Ur*eigUS[1])*s3US,
                    -(c55Ur*s1+c45Ur*s2)*eigUT[2]-(c55Ur*eigUT[0]+c45Ur*eigUT[1])*s3UT],
                   [-(c45Ur*s1+c44Ur*s2)*eigUP[2]-(c45Ur*eigUP[0]+c44Ur*eigUP[1])*s3UP,
                    -(c45Ur*s1+c44Ur*s2)*eigUS[2]-(c45Ur*eigUS[0]+c44Ur*eigUS[1])*s3US,
                    -(c45Ur*s1+c44Ur*s2)*eigUT[2]-(c45Ur*eigUT[0]+c44Ur*eigUT[1])*s3UT],
                   [eigUP[2], eigUS[2], eigUT[2]]])

    XL = np.array([[eigLP[0], eigLS[0], eigLT[0]],
                   [eigLP[1], eigLS[1], eigLT[1]],
                   [-(c13Lr*eigLP[0]+c36Lr*eigLP[1])*s1-(c23Lr*eigLP[1]+c36Lr*eigLP[0])*s2-c33Lr*eigLP[2]*s3LP,
                    -(c13Lr*eigLS[0]+c36Lr*eigLS[1])*s1-(c23Lr*eigLS[1]+c36Lr*eigLS[0])*s2-c33Lr*eigLS[2]*s3LS,
                    -(c13Lr*eigLT[0]+c36Lr*eigLT[1])*s1-(c23Lr*eigLT[1]+c36Lr*eigLT[0])*s2-c33Lr*eigLT[2]*s3LT]])

    YL = np.array([[-(c55Lr*s1+c45Lr*s2)*eigLP[2]-(c55Lr*eigLP[0]+c45Lr*eigLP[1])*s3LP,
                    -(c55Lr*s1+c45Lr*s2)*eigLS[2]-(c55Lr*eigLS[0]+c45Lr*eigLS[1])*s3LS,
                    -(c55Lr*s1+c45Lr*s2)*eigLT[2]-(c55Lr*eigLT[0]+c45Lr*eigLT[1])*s3LT],
                   [-(c45Lr*s1+c44Lr*s2)*eigLP[2]-(c45Lr*eigLP[0]+c44Lr*eigLP[1])*s3LP,
                    -(c45Lr*s1+c44Lr*s2)*eigLS[2]-(c45Lr*eigLS[0]+c44Lr*eigLS[1])*s3LS,
                    -(c45Lr*s1+c44Lr*s2)*eigLT[2]-(c45Lr*eigLT[0]+c44Lr*eigLT[1])*s3LT],
                   [eigLP[2], eigLS[2], eigLT[2]]])

    iXU = inverse3(XU)
    iYU = inverse3(YU)

    XUL = mult3(iXU,XL)
    YUL = mult3(iYU,YL)

    XpY = add3(XUL,YUL)
    XmY = sub3(XUL,YUL)

    Tran = inverse3(XpY)
    Refl = mult3(XmY,Tran)
    Tran = mult3(2.0,Tran)

    return(Refl[0][0])
